In [1]:
import sys
sys.path.append('../xrun')

In [2]:
import re

from pathlib import Path

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

sns.set(style="whitegrid")

from IPython.core.display import HTML
from sklearn.metrics import pairwise_distances

from xrun.data.run_info import RunInfo

In [3]:
def load_cost_from_file(file_path: Path):
    if file_path.exists():
            with open(file_path, "r") as f:
                return float(f.read())
    else:
        # print(f"Warning: {file_path} not found!")
        pass
    return None

def get_algorithm_name(run_info):
    if run_info.algorithm == "basic-clustering":
        return "StreamKM++ (buggy)"
    elif run_info.algorithm == "sensitivity-sampling":
        return "Sensitivity Sampling"
    elif run_info.algorithm == "group-sampling":
        return "Group Sampling"
    elif run_info.algorithm == "bico":
        return "BICO"
    elif run_info.algorithm == "stream-km++":
        return "StreamKM++"
    elif run_info.algorithm == "ray-maker":
        return "Ray Maker"
    return "Unknown"

def get_dataset_print_name(run_info):
    d = {
        "census": "Census",
        "censuslowd": "Census+PCA",
        "covertype": "Covertype",
        "covertypelowd": "Covertype+PCA",
        "tower": "Tower",
        "hardinstanceb1": "Benchmark",
        "hardinstanceb15": "Benchmark-1.5",
        "hardinstanceb2": "Benchmark-2.0",
        "oldhardinstanceb1": "Old Benchmark",
        "oldhardinstanceb2": "Old Benchmark-2.0",
        "caltech101": "Caltech",
        "caltech101lowd": "Caltech+PCA",
        "oldcaltech101": "Old Caltech",
        "nytimes100d": "NYTimes",
        "nytimespcalowd": "NYTimes+PCA",
    }
    if run_info.dataset in d:
        return d[run_info.dataset]
    return run_info.dataset
    

def get_costs(file_paths):
    costs = []
    for index, run_file_path in enumerate(file_paths):
        run_info = RunInfo.load_json(run_file_path)
        if "hardinstance" in run_info.dataset and run_info.k > 40:
            continue
        real_cost = load_cost_from_file(run_file_path.parent / "real_cost.txt")
        coreset_cost = load_cost_from_file(run_file_path.parent / "coreset_cost.txt")
        distortion = None
        if real_cost is not None and coreset_cost is not None:
            distortion = max(float(real_cost/coreset_cost), float(coreset_cost/real_cost))
        costs.append({
            "dataset": get_dataset_print_name(run_info),
            "algorithm": get_algorithm_name(run_info),
            "k": run_info.k,
            "coreset_size": run_info.m,
            "coreset_size_factor": int(run_info.m / run_info.k),
            "running_time": int(run_info.duration_secs),
            "real_cost": real_cost,
            "coreset_cost": coreset_cost,
            "distortion": distortion,
            "run_file_path": str(run_file_path),
        })
    df_data = pd.DataFrame(costs)
    #return df_data.groupby(["dataset", "algorithm", "k", "coreset_size"], as_index=False).head(10)
    return df_data

def aggregate_costs(df_costs: pd.DataFrame):
    group_columns = ["dataset", "algorithm", "k", "coreset_size", "coreset_size_factor"]
    df_top_k = df_costs.groupby(group_columns, as_index=False).head(10)
    df_aggr_costs = df_top_k.groupby(group_columns, as_index=False).agg(
        experiment_count=("algorithm", "count"),
        real_cost_mean=("real_cost", "mean"),
        real_cost_std=("real_cost", "std"),
        coreset_cost_mean=("coreset_cost", "mean"),
        coreset_cost_std=("coreset_cost", "std"),
        running_time_mean=("running_time", "mean"),
        running_time_std=("running_time", "std"),
        distortion_mean=("distortion", "mean"),
        distortion_median=("distortion", "median"),
        distortion_std=("distortion", "std"),
        distortion_max=("distortion", "max"),
    )
    df_aggr_costs["running_time"] = pd.to_timedelta(df_aggr_costs.running_time_mean, unit='s')
    df_aggr_costs["running_time_formatted"] = df_aggr_costs["running_time"].map(
        lambda x: f"{x.components.hours:02d}h {x.components.minutes:02d}m {x.components.seconds:02d}s"
    )
    return df_aggr_costs


def display_results_for(df_aggr_costs: pd.DataFrame, dataset_name: str, show_counts: bool=True, show_running_times:bool=True, show_costs: bool=True):
    pd.set_option('display.float_format', '{:.2e}'.format)
    
    display(HTML(f"<h2 style='border-bottom:solid 1px Black;padding-bottom:5px;'>Results for {dataset_name}</h2>"))
    df_filtered = df_aggr_costs[df_aggr_costs.dataset == dataset_name]
    
    html_str = ""
    
    if show_counts:
        df_counts = pd.pivot_table(df_filtered, values="experiment_count", index=["algorithm"], columns=["k"])
        df_counts = df_counts.rename_axis(None, axis=0).rename_axis(None, axis=1)
        html_str += f'<div style="border:solid 1px White; width:300px;float:left;">'
        html_str += f"<h4 style='text-align:center;'>Experiment Counts</h4>"
        html_str += df_counts.style.format(precision=0).to_html()
        html_str += "</div>"
    
    if show_running_times:
        df_run_times = pd.pivot_table(df_filtered, values="running_time_formatted", index=["algorithm"], columns=["k"], aggfunc=lambda x: x)
        df_run_times = df_run_times.rename_axis(None, axis=0).rename_axis(None, axis=1)

        html_str += f'<div style="border:solid 1px White; float:left;">'
        html_str += f"<h4 style='text-align:center;'>Average Running Times</h4>"
        html_str += df_run_times.to_html()
        html_str += "</div>"
        
    display(HTML(html_str))

    if show_costs:
        df_real_costs = pd.pivot_table(df_filtered, values="real_cost_mean", index=["algorithm"], columns=["k"])
        df_real_costs = df_real_costs.rename_axis(None, axis=0).rename_axis(None, axis=1)
        
        df_coreset_costs = pd.pivot_table(df_filtered, values="coreset_cost_mean", index=["algorithm"], columns=["k"])
        df_coreset_costs = df_coreset_costs.rename_axis(None, axis=0).rename_axis(None, axis=1)
        
        html_str = ""
        html_str += f'<div style="border:solid 1px #eee; float:left;">'
        html_str += f"<h4 style='text-align:center;'>Real costs</h4>"
        html_str += df_real_costs.to_html()
        html_str += "</div>"
        
        html_str += f'<div style="border:solid 1px #eee; float:left;">'
        html_str += f"<h4 style='text-align:center;'>Coreset costs</h4>"
        html_str += df_coreset_costs.to_html()
        html_str += "</div>"
        display(HTML(html_str))
    
    display(HTML(f'<h4>Distortions</h4>'))
    df_distortions = pd.pivot_table(df_filtered, values="distortion_max", index=["algorithm"], columns=["k"])
    df_distortions = df_distortions.rename_axis(None, axis=0).rename_axis(None, axis=1)
    display(df_distortions)
    
def add_combined_mean_std(df: pd.DataFrame, attr: str):
    def combiner(row) -> str:
        mean = row[f"{attr}_mean"]
        std =  row[f"{attr}_std"]
        return f"{mean:0.2f} ({std:0.3f})"
    df[f"{attr}_mean_std"] = df.apply(lambda x: combiner(x), axis=1)

In [4]:
data_results_dir = Path("../data/experiments-odin/")
run_files = list(data_results_dir.glob("**/*.json"))

In [5]:
len(run_files)

2025

In [6]:
df_cost_data = get_costs(run_files)

In [7]:
include_datasets = [
    'Caltech', 'Caltech+PCA', 'Census', 'Census+PCA', 'Covertype', 'Covertype+PCA', 
    'Tower', 'NYTimes', 'NYTimes+PCA', 
    # 'Benchmark-1.0', 'Benchmark-1.5', 'Benchmark-2.0',
    'Benchmark',
]
include_algorithms = [
    'StreamKM++', 'Group Sampling', 'Ray Maker', 'BICO', 'Sensitivity Sampling', 
    # 'StreamKM++ (buggy)'
]
df_cost_filtered = df_cost_data[df_cost_data.algorithm.isin(include_algorithms) & df_cost_data.dataset.isin(include_datasets)]
#df_cost_filtered = df_cost_data[df_cost_data.algorithm.isin(include_algorithms)]

In [8]:
df_aggr_costs = aggregate_costs(df_cost_filtered)

In [9]:
colors = sns.color_palette("tab10")
algorithm_colors = {algo: colors[i] for i, algo in enumerate(df_aggr_costs.algorithm.unique())}

In [10]:
df_aggr_costs["k_formatted"] = "$k$=" + df_aggr_costs["k"].astype(str)

In [11]:
add_combined_mean_std(df=df_aggr_costs, attr="distortion")

df_results_table = pd.pivot_table(
    data=df_aggr_costs,
    values="distortion_mean_std",
    index=["dataset", "k", "coreset_size"],
    columns=["algorithm"],
    aggfunc=lambda x: ' '.join(x)
)

df_results_table

algorithm                           BICO Group Sampling     Ray Maker  \
dataset   k   coreset_size                                              
Covertype 10  500           1.25 (0.013)   1.07 (0.010)  1.31 (0.025)   
              1000          1.16 (0.015)   1.05 (0.008)  1.22 (0.005)   
              2000          1.11 (0.000)   1.03 (0.011)  1.16 (0.003)   
          20  1000          1.26 (0.015)   1.10 (0.022)  1.34 (0.020)   
              2000          1.18 (0.003)   1.05 (0.010)  1.25 (0.008)   
              4000          1.11 (0.003)   1.03 (0.000)  1.22 (0.012)   
          30  1500          1.29 (0.008)   1.08 (0.009)  1.38 (0.011)   
              3000          1.16 (0.003)   1.05 (0.008)  1.26 (0.012)   
              6000          1.10 (0.001)   1.02 (0.011)  1.21 (0.014)   
          40  2000          1.27 (0.011)   1.09 (0.016)  1.38 (0.011)   
              4000          1.18 (0.004)   1.04 (0.007)  1.27 (0.011)   
              8000          1.09 (0.000)   1.03 (0.006)    1.22 (nan)   
          50  2500          1.28 (0.005)   1.09 (0.006)  1.38 (0.006)   
              5000          1.15 (0.002)   1.04 (0.004)  1.27 (0.004)   
              10000         1.07 (0.002)   1.02 (0.001)    1.22 (nan)   
Tower     20  1000             nan (nan)   1.09 (0.029)  1.44 (0.041)   
              2000             nan (nan)      nan (nan)     nan (nan)   
              4000             nan (nan)      nan (nan)  1.06 (0.006)   
              10000         1.03 (0.001)      nan (nan)  1.03 (0.002)   
          40  2000          1.19 (0.013)   1.10 (0.014)     nan (nan)   
              4000             nan (nan)      nan (nan)  1.15 (0.006)   
              8000          1.07 (0.005)      nan (nan)     nan (nan)   
              20000            nan (nan)   1.01 (0.003)  1.03 (0.001)   
          60  3000             nan (nan)      nan (nan)     nan (nan)   
              6000          1.10 (0.002)      nan (nan)     nan (nan)   
              12000            nan (nan)      nan (nan)     nan (nan)   
              30000            nan (nan)   1.02 (0.005)  1.02 (0.001)   
          80  4000             nan (nan)   1.11 (0.010)     nan (nan)   
              8000             nan (nan)   1.06 (0.008)  1.13 (0.004)   
              16000         1.05 (0.001)   1.03 (0.005)     nan (nan)   
              40000         1.02 (0.001)   1.02 (0.003)     nan (nan)   
          100 5000             nan (nan)      nan (nan)     nan (nan)   
              10000         1.09 (0.008)      nan (nan)  1.12 (0.006)   
              20000            nan (nan)   1.03 (0.005)     nan (nan)   
              50000         1.01 (0.001)      nan (nan)     nan (nan)   

algorithm                  Sensitivity Sampling    StreamKM++  
dataset   k   coreset_size                                     
Covertype 10  500                  1.04 (0.017)  1.05 (0.010)  
              1000                 1.03 (0.011)  1.03 (0.003)  
              2000                   1.02 (nan)    1.01 (nan)  
          20  1000                 1.04 (0.007)  1.04 (0.007)  
              2000                 1.02 (0.007)    1.03 (nan)  
              4000                   1.01 (nan)    1.01 (nan)  
          30  1500                 1.03 (0.015)  1.05 (0.004)  
              3000                 1.01 (0.006)    1.03 (nan)  
              6000                   1.02 (nan)    1.01 (nan)  
          40  2000                 1.03 (0.015)  1.05 (0.003)  
              4000                 1.02 (0.003)  1.03 (0.001)  
              8000                   1.01 (nan)  1.01 (0.001)  
          50  2500                 1.04 (0.009)  1.05 (0.003)  
              5000                 1.01 (0.003)  1.02 (0.003)  
              10000                  1.01 (nan)    1.01 (nan)  
Tower     20  1000                    nan (nan)     nan (nan)  
              2000                    nan (nan)     nan (nan)  
              4000                    nan (nan)     nan (nan)  
              1000

In [16]:
add_combined_mean_std(df=df_aggr_costs, attr="running_time")

df_results_table = pd.pivot_table(
    data=df_aggr_costs,
    values="running_time_mean_std",
    index=["dataset", "k", "coreset_size_factor"],
    columns=["algorithm"],
    aggfunc=lambda x: ' '.join(x)
)

df_results_table

algorithm                                   BICO    Group Sampling  \
dataset   k   coreset_size_factor                                    
Covertype 10  50                    7.25 (1.165)     51.00 (2.211)   
              100                   8.14 (0.900)     56.00 (0.816)   
              200                   9.00 (1.414)     53.67 (1.211)   
          20  50                    8.50 (0.926)     84.40 (4.648)   
              100                   8.57 (0.535)     87.43 (2.070)   
              200                  11.00 (1.414)     90.00 (5.657)   
          30  50                    8.50 (0.535)    116.75 (3.059)   
              100                   9.43 (0.535)    119.00 (1.000)   
              200                  12.00 (0.000)    116.50 (0.707)   
          40  50                    8.00 (0.756)    148.25 (4.301)   
              100                   9.86 (0.690)    151.43 (1.512)   
              200                  14.00 (0.000)    147.00 (2.828)   
          50  50                    9.75 (1.165)    179.00 (8.586)   
              100                  10.71 (0.488)    178.29 (6.800)   
              200                  15.50 (0.707)    176.50 (0.707)   
Tower     20  50                    7.60 (2.066)   265.70 (12.000)   
              100                   7.80 (1.989)   268.10 (12.565)   
              200                   7.40 (1.174)   268.50 (12.295)   
              500                   7.70 (1.829)    265.30 (7.732)   
          40  50                    7.80 (1.874)    546.40 (9.913)   
              100                   6.60 (0.843)   552.40 (14.401)   
              200                   7.70 (1.494)   548.20 (13.903)   
              500                   9.60 (1.713)   547.20 (16.376)   
          60  50                    6.30 (0.949)   831.10 (19.122)   
              100                   6.60 (1.350)   831.70 (13.524)   
              200                   7.10 (1.101)   842.30 (12.685)   
              500                  14.20 (4.264)   835.20 (16.858)   
          80  50                    7.30 (1.889)  1125.30 (20.249)   
              100                   7.60 (1.713)  1125.30 (12.019)   
              200                   8.60 (1.647)  1128.80 (21.149)   
              500                  20.10 (4.175)  1122.40 (13.656)   
          100 50                    6.90 (1.792)  1419.10 (22.083)   
              100                   7.50 (1.354)  1434.20 (21.343)   
              200                  10.50 (0.850)  1427.80 (14.559)   
              500                  28.50 (7.200)   1444.20 (8.892)   

algorithm                                Ray Maker Sensitivity Sampling  \
dataset   k   coreset_size_factor                                         
Covertype 10  50                     58.25 (1.581)        44.71 (3.251)   
              100                    62.57 (3.867)        44.14 (2.116)   
              200                    75.00 (2.828)        47.50 (4.950)   
          20  50                     66.29 (1.704)        69.29 (3.094)   
              100                    71.14 (3.934)        69.00 (2.449)   
              200                    83.50 (2.121)        68.50 (0.707)   
          30  50                     78.57 (2.878)        91.57 (1.988)   
              100                    82.29 (6.800)        93.67 (1.966)   
              200                    84.00 (8.485)        96.50 (0.707)   
          40  50                     89.57 (2.070)       122.57 (4.392)   
              100                    94.57 (1.397)       123.00 (2.000)   
              200                   106.00 (1.414)       124.50 (2.121)   
          50  50                     96.71 (5.823)       141.86 (2.478)   
              100                   105.71 (0.488)       145.17 (1.722)   
              200                   114.00 (1.414)       145.00 (2.828)   
Tower     20  50                     66.00 (2.582)        68.20 (1.549)   
              100                    90.80 (4.662)        7

In [15]:
df_results_table = pd.pivot_table(
    data=df_aggr_costs,
    values="distortion_mean_std",
    index=["dataset", "k", "coreset_size_factor"],
    columns=["algorithm"],
    aggfunc=lambda x: ' '.join(x)
)

df_results_table

algorithm                                  BICO Group Sampling     Ray Maker  \
dataset   k   coreset_size_factor                                              
Covertype 10  50                   1.25 (0.013)   1.07 (0.010)  1.31 (0.025)   
              100                  1.16 (0.015)   1.05 (0.008)  1.22 (0.005)   
              200                  1.11 (0.000)   1.03 (0.011)  1.16 (0.003)   
          20  50                   1.26 (0.015)   1.10 (0.022)  1.34 (0.020)   
              100                  1.18 (0.003)   1.05 (0.010)  1.25 (0.008)   
              200                  1.11 (0.003)   1.03 (0.000)  1.22 (0.012)   
          30  50                   1.29 (0.008)   1.08 (0.009)  1.38 (0.011)   
              100                  1.16 (0.003)   1.05 (0.008)  1.26 (0.012)   
              200                  1.10 (0.001)   1.02 (0.011)  1.21 (0.014)   
          40  50                   1.27 (0.011)   1.09 (0.016)  1.38 (0.011)   
              100                  1.18 (0.004)   1.04 (0.007)  1.27 (0.011)   
              200                  1.09 (0.000)   1.03 (0.006)    1.22 (nan)   
          50  50                   1.28 (0.005)   1.09 (0.006)  1.38 (0.006)   
              100                  1.15 (0.002)   1.04 (0.004)  1.27 (0.004)   
              200                  1.07 (0.002)   1.02 (0.001)    1.22 (nan)   
Tower     20  50                      nan (nan)   1.09 (0.029)  1.44 (0.041)   
              100                     nan (nan)      nan (nan)     nan (nan)   
              200                     nan (nan)      nan (nan)  1.06 (0.006)   
              500                  1.03 (0.001)      nan (nan)  1.03 (0.002)   
          40  50                   1.19 (0.013)   1.10 (0.014)     nan (nan)   
              100                     nan (nan)      nan (nan)  1.15 (0.006)   
              200                  1.07 (0.005)      nan (nan)     nan (nan)   
              500                     nan (nan)   1.01 (0.003)  1.03 (0.001)   
          60  50                      nan (nan)      nan (nan)     nan (nan)   
              100                  1.10 (0.002)      nan (nan)     nan (nan)   
              200                     nan (nan)      nan (nan)     nan (nan)   
              500                     nan (nan)   1.02 (0.005)  1.02 (0.001)   
          80  50                      nan (nan)   1.11 (0.010)     nan (nan)   
              100                     nan (nan)   1.06 (0.008)  1.13 (0.004)   
              200                  1.05 (0.001)   1.03 (0.005)     nan (nan)   
              500                  1.02 (0.001)   1.02 (0.003)     nan (nan)   
          100 50                      nan (nan)      nan (nan)     nan (nan)   
              100                  1.09 (0.008)      nan (nan)  1.12 (0.006)   
              200                     nan (nan)   1.03 (0.005)     nan (nan)   
              500                  1.01 (0.001)      nan (nan)     nan (nan)   

algorithm                         Sensitivity Sampling    StreamKM++  
dataset   k   coreset_size_factor                                     
Covertype 10  50                          1.04 (0.017)  1.05 (0.010)  
              100                         1.03 (0.011)  1.03 (0.003)  
              200                           1.02 (nan)    1.01 (nan)  
          20  50                          1.04 (0.007)  1.04 (0.007)  
              100                         1.02 (0.007)    1.03 (nan)  
              200                           1.01 (nan)    1.01 (nan)  
          30  50                          1.03 (0.015)  1.05 (0.004)  
              100                         1.01 (0.006)    1.03 (nan)  
              200                           1.02 (nan)    1.01 (nan)  
          40  50                          1.03 (0.015)  1.05 (0.003)  
              100                         1.02 (0.003)  1.03 (0.001)  
              200                           1.01 (nan)  1.01 (0.001)  
          50  50                          1.0